## Local Import

* working localy trim the data to ensure you can work with it locally <br\>*(my computer does not have sufficient local memory to join all tables so I use appropriate attributes)*

attempted stratagy (merge pd_df until memory bound then print to file)

In [1]:
import json, csv, os
import pandas as pd

def JSON_to_dataFrame(file): 
    
    fp = open(file, encoding="utf8")
    json_obj = [ json.loads(x) for x in fp.readlines() ] 
    df = pd.DataFrame(json_obj)
    fp.close
    return df


In [2]:
root_path = "/Users/2015mbp16gb256gb/Documents/school/Fall_2019/4502-Data_Mining/Group_Project/Group1/yelp_dataset/"
rbu_path = root_path + "rbu_merge.json"

In [3]:
%%time

review_df = JSON_to_dataFrame(root_path + "review.json")
business_df = JSON_to_dataFrame(root_path + "business.json")
user_df = JSON_to_dataFrame(root_path + "user.json")
# tip_df = JSON_to_dataFrame(root_path + "tip.json")                     /not used
# checkin_df = JSON_to_dataFrame(root_path + "checkin.json")             /not used

CPU times: user 2min 49s, sys: 2min 5s, total: 4min 55s
Wall time: 5min 34s


*********
### Merging Files
* paired the information down by combining the review business and user json files in to one merged csv file for processing (rbu_merged.csv) (deleted the other files and will be using this one to save space on my computer, if we need additional columns we will have to run this over adding the column)

In [4]:
# utility function to make a dict of columns with table name added
# used for the renaming of column names
def make_columns(lst, base):
    ret = {}
    for i in lst:
        ret[i] = base + '_' + i
    return ret

In [5]:
# return dataframe with column names updated to "table + _column_name"
def process_df(a_df, table): 

    col_list = [i for i in a_df]
    new_dict = make_columns(col_list, table)
    return a_df.rename(columns=new_dict)

In [6]:
%%time

# review col = ['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text', 'useful', 'user_id']

# ***Update review_df*** 
review_drop = ['cool','funny']
review_df = review_df.drop(review_drop, axis=1)

review_df = process_df(review_df, 'review')

CPU times: user 6.15 s, sys: 20.5 s, total: 26.6 s
Wall time: 36 s


In [7]:
%%time

# business col = 
# ['address','attributes','business_id','categories','city','hours','is_open',              
# 'latitude','longitude','name','postal_code','review_count','stars','state']                              

# ***Update business_df***
business_drop = ['address', 'attributes', 'categories','city', 'postal_code']
business_df = business_df.drop(business_drop, axis=1)

business_df = process_df(business_df, 'business')

CPU times: user 602 ms, sys: 1 s, total: 1.6 s
Wall time: 2.03 s


In [8]:
%%time

rb_merge = pd.merge(review_df, business_df, left_on='review_business_id', right_on='business_business_id', how='inner')

CPU times: user 29.4 s, sys: 1.63 s, total: 31 s
Wall time: 31.2 s


In [9]:
%%time

# user col = 
# ['average_stars','compliment_cool','compliment_cute','compliment_funny','compliment_hot','compliment_list',           
# 'compliment_more','compliment_note','compliment_photos','compliment_plain','compliment_profile','compliment_writer', 
# 'cool','elite','fans','friends','funny','name','review_count','useful','user_id','yelping_since']   

# ***Update _user_df***
user_drop = ['average_stars','compliment_cool','compliment_cute','compliment_funny','compliment_hot','compliment_list','compliment_more','compliment_note','compliment_photos','compliment_plain','compliment_profile','compliment_writer','cool', 'elite', 'fans','friends','funny']
user_df = user_df.drop(user_drop, axis=1)

user_df = process_df(user_df, 'user')

CPU times: user 3.5 s, sys: 6.57 s, total: 10.1 s
Wall time: 10.8 s


In [10]:
%%time

rbu_merge = pd.merge(rb_merge, user_df, left_on='review_user_id', right_on='user_user_id', how='inner')

CPU times: user 25 s, sys: 7.73 s, total: 32.7 s
Wall time: 34.2 s


In [11]:
%%time

rbu_drop = ['review_business_id','review_user_id', 'business_hours']
rbu_merge = rbu_merge.drop(rbu_drop, axis=1)


CPU times: user 19.2 s, sys: 4.76 s, total: 24 s
Wall time: 25.1 s


In [12]:
for col in rbu_merge: 
    print(col)

review_date
review_review_id
review_stars
review_text
review_useful
business_business_id
business_is_open
business_latitude
business_longitude
business_name
business_review_count
business_stars
business_state
user_name
user_review_count
user_useful
user_user_id
user_yelping_since


*********
### Writing Merged DataFrame Back To Json

In [13]:
def df_to_json(df, path):
    new_dict = {}
    num_col = len([i for i in df])

    with open(r'{}'.format(path), 'w') as writer: 

        for row in df.itertuples():
            
            for idx, col in enumerate(df): 
                if type(row[idx+1]) == int: 
                    new_dict[col] = int(row[idx+1])
                if type(row[idx+1]) == float: 
                    new_dict[col] = float(row[idx+1])
                else: 
                    new_dict[col] = str(row[idx+1])
                
            writer.write(json.dumps(new_dict) + '\n')
            new_dict = {}         



In [14]:
%%time

df_to_json(rbu_merge, rbu_path)

CPU times: user 5min 38s, sys: 24.4 s, total: 6min 2s
Wall time: 6min 7s


In [15]:
%%time

rbu_complete_df = JSON_to_dataFrame(rbu_path)

CPU times: user 3min 54s, sys: 11min 31s, total: 15min 25s
Wall time: 31min 39s


In [18]:
rbu_complete_df.isnull().sum()

business_business_id     0
business_is_open         0
business_latitude        0
business_longitude       0
business_name            0
business_review_count    0
business_stars           0
business_state           0
review_date              0
review_review_id         0
review_stars             0
review_text              0
review_useful            0
user_name                0
user_review_count        0
user_useful              0
user_user_id             0
user_yelping_since       0
dtype: int64

## Unit Test

In [19]:
test_in_path = root_path + 'test.json' 

test_df = JSON_to_dataFrame(test_in_path)

In [20]:
%%time

test_out_path = root_path + 'test_out.json' 

parse_DataFrame1(test_df, test_out_path)

CPU times: user 3.91 ms, sys: 15.6 ms, total: 19.5 ms
Wall time: 34.6 ms


In [25]:
test2_df = JSON_to_dataFrame(test_out_path)

In [55]:
def test_DataFrame_to_Json(a_df_1, a_df_2): 
    
    df_list_1 = []
    df_list_2 = []
    catch_list = []
    
    for row in a_df_1.itertuples():
            
            for idx, col in enumerate(a_df_1):
                df_list_1.append((type(row[idx]), col))
                
    for row_2 in a_df_2.itertuples():
            
            for idx_2, col_2 in enumerate(a_df_2):
                df_list_2.append((type(row[idx_2]), col_2))
                
    for x,y in zip(df_list_1, df_list_2): 
        if x != y: 
            catch_list.append((x, y))
            
    for i in catch_list: 
        print(i)             # dataFrams have different types

In [ ]:
# smaller scale test on business set
test_DataFrame_to_Json(rbu_merge, rbu_complete_df)

In [ ]:
# test on full merged data set
test_DataFrame_to_Json(rbu_merge, rbu_complete_df)

## Functions to check file given error message

In [ ]:
check_input_output(test_in_path, test_out_path, log_path)

In [ ]:
def check_input_output(a_test_in_path, a_test_out_path, a_log_path):
    line_count = 0
    char_count = 0
    save1 = ""
    save2 = []
    cnt1 = 0
    cnt2 = 0
    
    with open(a_test_in_path, 'r') as read1, open(a_test_out_path, 'r') as read2: 
        for x, y in zip(read1, read2):
            x = x.strip()
            y = y.strip()
            
            in_line = x.split()
            out_line = y.split()
            for word1, word2 in zip(in_line, out_line):
                
                save2.append(word1)
                
                if cnt1 > 5:
                    save2.pop(0)
                cnt1 = cnt1 + 1

            
                word_list = list(word2)
                
                for char in word_list: 
                    save1 = save1 + char
                    if cnt2 > 70: 
                        save1 = save1[1:]
                    
                    if char_count == 139: 
                        print(save1)
                        print(" ".join(save2))
                        
                    char_count = char_count + 1
                    cnt2 += 1
                
            line_count = line_count + 1


            

In [ ]:
with open(test_out_path, 'r') as reader: 
    for i in reader: 
        print(i)

****
## GCP Import
* login cradentaials provided to work with yelp_db database

In [ ]:
import sys
!{sys.executable} -m pip install PyMySQL

### MySql Server is nessisary to connect to GCP
* https://dev.mysql.com/downloads/mysql/

In [ ]:
import pymysql, os, sys, json
from IPython.display import clear_output
import multiprocessing as mp

In [ ]:
# read json file
file = "Group1/review.json"

json_data = open(file, encoding="utf8")
json_obj = [json.loads(x) for x in json_data.readlines()]


In [ ]:
num = len(json_obj)
print(num)

In [ ]:
# validate data before inserting
# TODO: check if string is the right type 
# TODO: check for duplicate data
# TODO: check for bad formats (date)

def validate_string(val): 
    if val != None: 
            if type(val) is int: 
                return str(val).encode('utf-8')
            else: 
                return val
            

### Processing takes to long so I used multiprocessing to speed up importing to host
* executing insert into gcp is atomic

this will take over 4 days to import one table!

In [ ]:
def insert_review(cursor, item, lock): 
          
    review_id = validate_string(item['review_id'])
    business_id = validate_string(item['business_id'])
    cool = validate_string(item['cool'])
    date = validate_string(item['date'])
    funny = validate_string(item['funny'])
    stars = validate_string(item['stars'])
    text = validate_string(item['text'])
    useful = validate_string(item['useful'])
    user_id = validate_string(item['user_id'])
    
    query = "INSERT INTO review(review_id,business_id,cool,date,funny,stars,text,useful,user_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    args = (review_id, business_id, cool, date, funny, stars, text, useful, user_id)
    lock.acquire()
    cursor.execute(query, args)
    lock.release()
    
    clear_output()
    cnt.value = cnt.value + 1
    print(cnt.value)
    
    

In [ ]:
# connect to GCP MySql db insert data into review table
con = pymysql.connect(host = '104.198.65.208', user='root', password = 'hackme', db = 'yelp_db')

cnt = mp.Value('i', 0)
lock = mp.Lock()
pool = mp.Pool()


try: 
    with con.cursor() as cursor:
        
        # parse json data to SQL
        # use map to update each row in GCP
        result = pool.map([ insert_review(cursor, item, lock) for item in json_obj ])
            
finally:
    con.commit()
    con.close()
    